In [10]:
import cv2
import mediapipe as mp
import time
import math

In [11]:
def find_distance(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [12]:
def find_angle(x1, y1, x2, y2):
    theta = math.acos((y2-y1)*(-y1) / (math.sqrt((x2-x1)**2 + (y2-y1)**2) * y1))
    degree = int(180/math.pi) * theta

    return degree

In [13]:
def send_warning(x):
    pass

In [14]:
# Initilize frame counters.
good_frames = 0
bad_frames = 0

# Font type.
font = cv2.FONT_HERSHEY_SIMPLEX

# Colors.
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [15]:
mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose()

cap = cv2.VideoCapture(0)

# Meta
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)

I0000 00:00:1703477189.317154       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.8.101), renderer: AMD Radeon R9 M370X OpenGL Engine


In [16]:
# norm_coordinate = pose.process(image).pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.<SPECIFIC_LANDMARK>].coordinate

In [17]:
print('Processing..')

if not cap.isOpened():
    print('Cannot open camera.')
    exit()

while True:
    # Capture frames.
    success, image = cap.read()
    if not success:
        print("Error: Failed to capture frame.")
        break
    
    start = time.time()
    
    # Display the frame
    cv2.imshow('Camera Feed', image)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    if not success:
        print("Null.Frames")
        break
    
    # Get height and width.
    h, w = image.shape[:2]

    # Convert the BGR image to RGB.
    imgRBG = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Improve performance
    image.flags.writeable = False

    # Process the image.
    keypoints = pose.process(imgRBG)
    
    # # Improve performance
    # image.flags.writeable = True

    # # Convert the image back to BGR.
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Use lm and lmPose as representative of the following methods.
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    if lm:
        print(lm)
        mp_draw.draw_landmarks(image, lm, mp_pose.POSE_CONNECTIONS)

    
    
print('Finished.')
cap.release()
cv2.destroyAllWindows()

Processing..
landmark {
  x: 0.8996199
  y: 0.90290093
  z: -0.27464968
  visibility: 0.97885746
}
landmark {
  x: 0.8664839
  y: 0.84029514
  z: -0.22320776
  visibility: 0.97080743
}
landmark {
  x: 0.868761
  y: 0.8199358
  z: -0.22393472
  visibility: 0.97254825
}
landmark {
  x: 0.8715963
  y: 0.80019706
  z: -0.22420765
  visibility: 0.9788614
}
landmark {
  x: 0.863388
  y: 0.88536054
  z: -0.20718887
  visibility: 0.9673432
}
landmark {
  x: 0.86288434
  y: 0.8963278
  z: -0.20719399
  visibility: 0.9649195
}
landmark {
  x: 0.8623448
  y: 0.9064445
  z: -0.20814843
  visibility: 0.97209066
}
landmark {
  x: 0.8957067
  y: 0.759598
  z: 0.053971563
  visibility: 0.97070956
}
landmark {
  x: 0.8805503
  y: 0.8806546
  z: 0.13770267
  visibility: 0.94404316
}
landmark {
  x: 0.94541097
  y: 0.86345047
  z: -0.16985545
  visibility: 0.87931323
}
landmark {
  x: 0.93845004
  y: 0.9213472
  z: -0.14985956
  visibility: 0.85522133
}
landmark {
  x: 1.1045425
  y: 0.57154334
  z: 0.04

In [18]:
# # Acquire the landmark coordinates.
#         # Once aligned properly, left or right should not be a concern.
#         # Left shoulder.
#         l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
#         l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
#         # Right shoulder
#         r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
#         r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
#         # Left ear.
#         l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
#         l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
#         # Left hip.
#         l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
#         l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

#         # Calculate distance between left shoulder and right shoulder points.
#         offset = find_distance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

#         # Assist to align the camera to point at the side view of the person.
#         # Offset threshold 30 is based on results obtained from analysis over 100 samples.
#         if offset < 100:
#             cv2.putText(image, str(int(offset)) + ' Aligned', (w - 150, 30), font, 0.9, green, 2)
#         else:
#             cv2.putText(image, str(int(offset)) + ' Not Aligned', (w - 150, 30), font, 0.9, red, 2)

#         # Calculate angles.
#         neck_inclination = find_angle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
#         torso_inclination = find_angle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

#         # Draw landmarks.
#         cv2.circle(image, (l_shldr_x, l_shldr_y), 7, yellow, -1)
#         cv2.circle(image, (l_ear_x, l_ear_y), 7, yellow, -1)

#         # Let's take y - coordinate of P3 100px above x1,  for display elegance.
#         # Although we are taking y = 0 while calculating angle between P1,P2,P3.
#         cv2.circle(image, (l_shldr_x, l_shldr_y - 100), 7, yellow, -1)
#         cv2.circle(image, (r_shldr_x, r_shldr_y), 7, pink, -1)
#         cv2.circle(image, (l_hip_x, l_hip_y), 7, yellow, -1)

#         # Similarly, here we are taking y - coordinate 100px above x1. Note that
#         # you can take any value for y, not necessarily 100 or 200 pixels.
#         cv2.circle(image, (l_hip_x, l_hip_y - 100), 7, yellow, -1)

#         # Put text, Posture and angle inclination.
#         # Text string for display.
#         angle_text_string = 'Neck : ' + str(int(neck_inclination)) + '  Torso : ' + str(int(torso_inclination))

#         # Determine whether good posture or bad posture.
#         # The threshold angles have been set based on intuition.
#         if neck_inclination < 40 and torso_inclination < 10:
#             bad_frames = 0
#             good_frames += 1

#             cv2.putText(image, angle_text_string, (10, 30), font, 0.9, light_green, 2)
#             cv2.putText(image, str(int(neck_inclination)), (l_shldr_x + 10, l_shldr_y), font, 0.9, light_green, 2)
#             cv2.putText(image, str(int(torso_inclination)), (l_hip_x + 10, l_hip_y), font, 0.9, light_green, 2)

#             # Join landmarks.
#             cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), green, 4)
#             cv2.line(image, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), green, 4)
#             cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), green, 4)
#             cv2.line(image, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), green, 4)

#         else:
#             good_frames = 0
#             bad_frames += 1

#             cv2.putText(image, angle_text_string, (10, 30), font, 0.9, red, 2)
#             cv2.putText(image, str(int(neck_inclination)), (l_shldr_x + 10, l_shldr_y), font, 0.9, red, 2)
#             cv2.putText(image, str(int(torso_inclination)), (l_hip_x + 10, l_hip_y), font, 0.9, red, 2)

#             # Join landmarks.
#             cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), red, 4)
#             cv2.line(image, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), red, 4)
#             cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), red, 4)
#             cv2.line(image, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), red, 4)

#         # Calculate the time of remaining in a particular posture.
#         good_time = (1 / fps) * good_frames
#         bad_time =  (1 / fps) * bad_frames

#         # Pose time.
#         if good_time > 0:
#             time_string_good = 'Good Posture Time : ' + str(round(good_time, 1)) + 's'
#             cv2.putText(image, time_string_good, (10, h - 20), font, 0.9, green, 2)
#         else:
#             time_string_bad = 'Bad Posture Time : ' + str(round(bad_time, 1)) + 's'
#             cv2.putText(image, time_string_bad, (10, h - 20), font, 0.9, red, 2)

#         # If you stay in bad posture for more than 3 minutes (180s) send an alert.
#         if bad_time > 180:
#             send_warning()